In [1]:
import os
os.environ['JUPYTER_RUNTIME_DIR'] = os.path.expanduser('~/jupyter/runtime') 
os.makedirs(os.environ['JUPYTER_RUNTIME_DIR'], exist_ok=True)

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scipy.sparse as sp
from intervaltree import Interval, IntervalTree
import gtfparse
import os

# For LSI (TF-IDF + SVD)
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from scipy.io import mmwrite, mmread # Useful if dealing with matrix market files, though not directly needed for h5


In [2]:
# --- Configuration ---
DATA_DIR = "../10x_pbmc"
ATAC_H5_FILE = os.path.join(DATA_DIR, "atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5")
GTF_FILE = os.path.join(DATA_DIR, "Homo_sapiens.GRCh37.82.gtf")
ATAC_META_FILE = os.path.join(DATA_DIR, "atac_v1_pbmc_10k_singlecell.csv")
RNA_H5AD_FILE = os.path.join(DATA_DIR, "pbmc_10k_v3.h5ad") # Assumed pre-processed h5ad file

# Gene Activity Calculation Parameters
UPSTREAM_KB = 2000
SEQ_LEVELS = [str(i) for i in range(1, 23)] + ["X", "Y"]


In [3]:
# --- 1. Gene Activity Quantification ---
print("1. Calculating Gene Activity Matrix...")

peak_matrix_file, gtf_file, seq_levels, upstream = ATAC_H5_FILE, GTF_FILE, SEQ_LEVELS, UPSTREAM_KB

"""
Calculates a gene activity matrix from a peak matrix and GTF file.

Args:
    peak_matrix_file (str): Path to the 10x H5 peak matrix file.
    gtf_file (str): Path to the GTF annotation file.
    seq_levels (list): List of chromosomes/contigs to include.
    upstream (int): Number of base pairs upstream of TSS to include.

Returns:
    anndata.AnnData: An AnnData object with genes as vars and cells as obs,
                        containing the gene activity matrix in X.
                        Also includes peak coordinates in .uns['peak_intervals']
                        and the original peak matrix AnnData in .uns['peak_adata']
                        for reference.
"""
print(f"  Loading peak matrix: {peak_matrix_file}")
adata_peaks = sc.read_10x_h5(peak_matrix_file)
adata_peaks.var_names_make_unique()

1. Calculating Gene Activity Matrix...
  Loading peak matrix: ../10x_pbmc/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5


In [4]:
print(f"  Loading and parsing GTF: {gtf_file}")
# Using gtfparse to read the GTF
gtf = gtfparse.read_gtf(gtf_file)

# Filter for genes on desired chromosomes
gtf_genes = gtf[gtf['feature'] == 'gene']
gtf_genes = gtf_genes[gtf_genes['seqname'].isin(seq_levels)]
print(f"  Found {len(gtf_genes)} genes on specified seq levels.")

  Loading and parsing GTF: ../10x_pbmc/Homo_sapiens.GRCh37.82.gtf


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'havana_transcript', 'havana_transcript_version', 'tag', 'exon_number', 'exon_id', 'exon_version', 'ccds_id', 'protein_id', 'protein_version']


ValueError: Wrong number of dimensions. values.ndim > ndim [2 > 1]

In [4]:
print(f"  Loading and parsing GTF: {gtf_file}")
# Read the GTF file directly with pandas
gtf = pd.read_csv(gtf_file, sep='\t', comment='#', 
                  names=['seqname', 'source', 'feature', 'start', 'end', 
                         'score', 'strand', 'frame', 'attribute'])

  Loading and parsing GTF: ../10x_pbmc/Homo_sapiens.GRCh37.82.gtf


/tmp/ipykernel_3474622/2105413042.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv(gtf_file, sep='\t', comment='#',


In [5]:
# Parse the attributes column (which contains key-value pairs)
def parse_attributes(attr_str):
    if pd.isna(attr_str):
        return {}
    attrs = {}
    for attr in attr_str.split(';'):
        if attr.strip():
            try:
                key, value = attr.strip().split(' ', 1)
                attrs[key] = value.strip('"')
            except ValueError:
                continue
    return attrs


In [6]:
# Apply the function to extract attributes
attr_dicts = gtf['attribute'].apply(parse_attributes)

# Extract commonly used attributes
common_attrs = ['gene_id', 'gene_name', 'gene_type', 'transcript_id']
for attr in common_attrs:
    gtf[attr] = attr_dicts.apply(lambda x: x.get(attr, None))


: 

In [ ]:
# Filter for genes on desired chromosomes
gtf_genes = gtf[gtf['feature'] == 'gene']
gtf_genes = gtf_genes[gtf_genes['seqname'].isin(seq_levels)]
print(f"  Found {len(gtf_genes)} genes on specified seq levels.")

  Loading and parsing GTF: ../10x_pbmc/Homo_sapiens.GRCh37.82.gtf


/tmp/ipykernel_3474520/3091208214.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv(gtf_file, sep='\t', comment='#',


: 

In [ ]:
# Create interval trees for peaks
peak_intervals = {}
peak_mapping = {} # To map peak names back to indices in adata_peaks.var
print("  Creating peak interval trees...")
for i, peak in enumerate(adata_peaks.var_names):
    try:
        chrom, coords = peak.split(':')
        start, end = map(int, coords.split('-'))
        if chrom not in peak_intervals:
            peak_intervals[chrom] = IntervalTree()
        peak_intervals[chrom][start:end] = i # Store index
        peak_mapping[peak] = i
    except ValueError:
        print(f"    Warning: Could not parse peak coordinates: {peak}")
        continue

# Create gene activity matrix (sparse)
gene_activity_matrix = sp.lil_matrix((len(gtf_genes), adata_peaks.n_obs), dtype=np.float32)
gene_names = []
gene_coords = {} # Store gene coords for reference

print("  Matching peaks to genes...")
processed_genes = 0
for idx, gene in gtf_genes.iterrows():
    gene_id = gene['gene_id']
    gene_name = gene['gene_name'] if 'gene_name' in gene and pd.notna(gene['gene_name']) else gene_id
    chrom = gene['seqname']
    start = gene['start']
    end = gene['end']
    strand = gene['strand']

    # Define gene body region (+ upstream)
    if strand == '+':
        region_start = max(0, start - upstream)
        region_end = end
    else: # strand == '-'
        region_start = start
        region_end = end + upstream

    gene_names.append(gene_name) # Use gene_name if available
    gene_coords[gene_name] = f"{chrom}:{region_start}-{region_end}"

    # Find overlapping peaks
    if chrom in peak_intervals:
        overlapping_peaks = peak_intervals[chrom][region_start:region_end]
        if overlapping_peaks:
            peak_indices = [p.data for p in overlapping_peaks]
            # Sum counts for these peaks across all cells
            gene_activity_vector = adata_peaks[:, peak_indices].X.sum(axis=1)
            # Add to the sparse matrix
            gene_activity_matrix[processed_genes, :] = gene_activity_vector.T # Assign the dense row

    processed_genes += 1
    if processed_genes % 1000 == 0:
        print(f"    Processed {processed_genes}/{len(gtf_genes)} genes...")

print("  Finalizing gene activity matrix...")
# Convert to CSR format for efficiency
gene_activity_matrix = gene_activity_matrix.tocsr()

# Create AnnData object for gene activity
adata_activity = ad.AnnData(gene_activity_matrix.T) # Cells x Genes
adata_activity.var_names = gene_names
adata_activity.obs_names = adata_peaks.obs_names

# Make gene names unique if duplicates exist
adata_activity.var_names_make_unique()

# Add peak info to .uns for potential future use
adata_activity.uns['peak_intervals'] = peak_intervals
adata_activity.uns['peak_adata_var_names'] = adata_peaks.var_names.tolist() # Store original peak names
adata_activity.uns['gene_coords'] = gene_coords

print("Gene Activity Calculation Complete.")


1. Calculating Gene Activity Matrix...
  Loading peak matrix: ../10x_pbmc/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5
  Loading and parsing GTF: ../10x_pbmc/Homo_sapiens.GRCh37.82.gtf


: 

In [ ]:
# --- 2. Object Setup and QC ---
print("\n2. Setting up AnnData Object and QC...")

# Create the main AnnData object using the peak matrix initially
# We will store gene activity in a layer
pbmc_atac = adata_peaks_orig.copy()
pbmc_atac.layers["activity"] = adata_activity[pbmc_atac.obs_names, :].X.T # Ensure cells match and shape is peaks x cells

# Load metadata
meta = pd.read_csv(ATAC_META_FILE, index_col=0)

# Align metadata with the AnnData object
common_barcodes = pbmc_atac.obs_names.intersection(meta.index)
print(f"  Found {len(common_barcodes)} barcodes common between matrix and metadata.")
pbmc_atac = pbmc_atac[common_barcodes, :].copy()
meta = meta.loc[common_barcodes, :]

# Add metadata to AnnData object
for col in meta.columns:
    pbmc_atac.obs[col] = meta[col]

# QC: Filter cells with fewer than 5K total counts in the ATAC data
# Seurat's nCount_ATAC corresponds to the sum of counts per cell in the *peak* matrix
sc.pp.calculate_qc_metrics(pbmc_atac, qc_vars=['atac'], percent_top=None, log1p=False, inplace=True) # calculates n_genes_by_counts, total_counts
# Rename 'total_counts' to match the vignette's 'nCount_ATAC' for clarity
pbmc_atac.obs['nCount_ATAC'] = pbmc_atac.obs['total_counts']

n_cells_before = pbmc_atac.n_obs
pbmc_atac = pbmc_atac[pbmc_atac.obs['nCount_ATAC'] > 5000, :]
print(f"  Filtered cells based on nCount_ATAC > 5000. Kept {pbmc_atac.n_obs} out of {n_cells_before} cells.")

# Add technology label
pbmc_atac.obs['tech'] = 'atac'

In [ ]:
# --- 3. Data Preprocessing (Gene Activity) ---
print("\n3. Preprocessing Gene Activity Matrix...")

# Create a temporary AnnData with activity matrix for processing
adata_activity_proc = ad.AnnData(pbmc_atac.layers["activity"].T) # Genes x Cells -> Cells x Genes
adata_activity_proc.obs_names = pbmc_atac.obs_names
adata_activity_proc.var_names = adata_activity.var_names # Use original gene names from calculation

# Basic processing for gene activity matrix - required for finding anchors
sc.pp.highly_variable_genes(adata_activity_proc, n_top_genes=4000, flavor='seurat_v3')
sc.pp.normalize_total(adata_activity_proc, target_sum=1e4)
sc.pp.log1p(adata_activity_proc)
sc.pp.scale(adata_activity_proc, max_value=10)

# Store the processed activity data back (or keep adata_activity_proc separate)
# For simplicity, let's store the HVG info and scaled data back into the main object
pbmc_atac.var['highly_variable_activity'] = adata_activity_proc.var['highly_variable']
# Note: Scanpy's scale overwrites .X. If needed later, store scaled data in a layer
# pbmc_atac.layers['activity_scaled'] = adata_activity_proc.X


In [ ]:
# --- 4. Data Preprocessing (Peak Matrix - LSI) ---
print("\n4. Preprocessing Peak Matrix (LSI)...")

# Filter peaks: Keep peaks present in at least 100 cells
min_cells_per_peak = 100
sc.pp.filter_genes(pbmc_atac, min_cells=min_cells_per_peak) # filter genes (peaks) based on cells
print(f"  Filtered peaks. Kept {pbmc_atac.n_vars} peaks present in > {min_cells_per_peak} cells.")

# LSI Implementation (TF-IDF followed by SVD)
print("  Calculating TF-IDF...")
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
# Needs features (peaks) x samples (cells)
tfidf_matrix = tfidf.fit_transform(pbmc_atac.X.T) # Transpose to get peaks x cells

# Reduce dimensionality using SVD (LSI)
n_components = 50 # As used in the vignette (dims 2 to 50 are often used, skipping first)
print(f"  Running TruncatedSVD (LSI) with {n_components} components...")
svd = TruncatedSVD(n_components=n_components, algorithm='arpack', random_state=42)
lsi_matrix = svd.fit_transform(tfidf_matrix) # Result is peaks x components

# The resulting components are analogous to PCA components.
# We need cell embeddings (cells x components).
# In LSI, cell embeddings are often derived differently, sometimes V.T * Sigma
# However, a common practical approach is to use the transform of the original matrix
# Let's project the cells onto the components found: cells x components
# Note: Seurat RunLSI V.T is the cell embedding (cells x components)
# lsi_cell_embeddings = tfidf_matrix.T @ svd.components_.T # Doesn't seem right dimensionally
lsi_cell_embeddings = svd.transform(tfidf_matrix.T) # Project cell vectors (tfidf) onto components

# Store LSI results in AnnData object (cells x components)
pbmc_atac.obsm['X_lsi'] = lsi_cell_embeddings

# Run UMAP on LSI components
print("  Running UMAP on LSI components...")
# Use components 2 to 50 (index 1 to 49) as often the first component captures technical variation/depth
sc.pp.neighbors(pbmc_atac, n_neighbors=30, use_rep='X_lsi', n_pcs=n_components, key_added='lsi_neighbors') # Use all components for neighbor graph
sc.tl.umap(pbmc_atac, min_dist=0.3, neighbors_key='lsi_neighbors')
# Store this UMAP separately
pbmc_atac.obsm['X_umap_lsi'] = pbmc_atac.obsm['X_umap']


In [ ]:
# --- 5. Load scRNA-seq Data ---
print("\n5. Loading pre-processed scRNA-seq data...")
try:
    pbmc_rna = sc.read_h5ad(RNA_H5AD_FILE)
    pbmc_rna.obs['tech'] = 'rna'
    print("  scRNA-seq data loaded successfully.")
    # Assume pbmc_rna has 'celltype' in .obs and 'X_umap' in .obsm
    # Ensure variable genes are marked if not already
    if 'highly_variable' not in pbmc_rna.var.columns:
         print("  Warning: 'highly_variable' not found in RNA data, calculating again...")
         sc.pp.highly_variable_genes(pbmc_rna, n_top_genes=4000, flavor='seurat_v3') # Use same params as activity
except FileNotFoundError:
    print(f"  Error: RNA data file not found at {RNA_H5AD_FILE}. Cannot proceed with integration.")
    pbmc_rna = None
except Exception as e:
    print(f"  Error loading RNA data: {e}. Cannot proceed with integration.")
    pbmc_rna = None

In [ ]:
# --- 6. Visualize Initial Embeddings ---
if pbmc_rna is not None:
    print("\n6. Visualizing initial UMAP embeddings...")
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    sc.pl.umap(pbmc_atac, color='tech', ax=axes[0], show=False, title="scATAC-seq (LSI UMAP)", use_raw=False, basis='umap_lsi')
    sc.pl.umap(pbmc_rna, color='celltype', ax=axes[1], show=False, title="scRNA-seq (RNA UMAP)", legend_loc='on data', legend_fontsize=8)

    plt.tight_layout()
    plt.show()


In [ ]:
# --- 7. Identify Anchors and Transfer Labels ---
# Scanpy uses 'ingest' for mapping/label transfer, conceptually similar to FindTransferAnchors + TransferData
if pbmc_rna is not None:
    print("\n7. Transferring Cell Type Labels using Scanpy Ingest...")

    # Ensure RNA data has PCA calculated, as ingest uses it by default
    if 'X_pca' not in pbmc_rna.obsm:
        print("  Calculating PCA for RNA data (required for ingest)...")
        sc.tl.pca(pbmc_rna, svd_solver='arpack', use_highly_variable=True)

    # --- Re-aligning ATAC data for Ingest ---
    # Ingest requires the query object (ATAC) to have data for the features used in the reference (RNA).
    # The vignette uses the 'ACTIVITY' assay (gene activity). We need an AnnData for ATAC
    # containing the *unscaled* log-normalized gene activity for the *variable genes* of the RNA reference.

    # 1. Get variable genes from RNA reference
    var_genes_rna = pbmc_rna.var_names[pbmc_rna.var['highly_variable']]
    print(f"  Using {len(var_genes_rna)} variable genes from RNA reference.")

    # 2. Create ATAC AnnData with log-normalized activity for these genes
    # Use the original activity matrix stored in pbmc_atac.layers['activity']
    common_genes = pbmc_atac.layers['activity'].var_names.intersection(var_genes_rna)
    print(f"  Found {len(common_genes)} common variable genes in ATAC activity matrix.")

    adata_atac_activity_for_ingest = ad.AnnData(pbmc_atac.layers['activity'][:, common_genes].T) # Cells x Genes
    adata_atac_activity_for_ingest.obs = pbmc_atac.obs.copy()
    adata_atac_activity_for_ingest.obsm['X_lsi'] = pbmc_atac.obsm['X_lsi'] # Carry over LSI for weight reduction

    # 3. Normalize and log-transform this specific matrix
    sc.pp.normalize_total(adata_atac_activity_for_ingest, target_sum=1e4)
    sc.pp.log1p(adata_atac_activity_for_ingest)

    # --- Run Ingest ---
    # reference.assay = "RNA", query.assay = "ACTIVITY" -> Use RNA PCA space and ATAC activity data
    # reduction = "cca" -> Ingest uses PCA space of reference by default
    # weight.reduction = pbmc.atac[["lsi"]] -> Pass LSI embedding via `obsm_key`
    print("  Running sc.tl.ingest...")
    sc.tl.ingest(
        adata_atac_activity_for_ingest, # Query (ATAC activity)
        pbmc_rna,                       # Reference (RNA)
        obs='celltype',                 # Column to transfer
        embedding_basis='pca',          # Map to reference PCA space
        obsm_key='X_lsi',               # Use ATAC LSI for weighting (analogous to weight.reduction)
        inplace=True                    # Add results to adata_atac_activity_for_ingest
    )

    # Transfer results back to the main ATAC object
    pbmc_atac.obs['predicted_id'] = adata_atac_activity_for_ingest.obs['celltype']
    # Ingest provides probabilities in .uns['ingest']['celltype_probabilities']
    # Find the max probability for each cell
    transfer_probabilities = adata_atac_activity_for_ingest.uns['ingest']['celltype_probabilities']
    pbmc_atac.obs['prediction_score_max'] = transfer_probabilities.max(axis=1)

    print("  Label transfer complete.")

    # --- Evaluate Transfer ---
    print("  Evaluating transfer prediction scores...")
    plt.figure(figsize=(6, 4))
    plt.hist(pbmc_atac.obs['prediction_score_max'], bins=50)
    plt.axvline(0.5, color='red', linestyle='--')
    plt.title("Prediction Score Distribution")
    plt.xlabel("Max Prediction Score")
    plt.ylabel("Frequency")
    plt.show()

    score_threshold = 0.5
    pass_threshold = pbmc_atac.obs['prediction_score_max'] > score_threshold
    print(f"  Cells passing threshold ({score_threshold}): {pass_threshold.sum()} / {pbmc_atac.n_obs}")

    # Filter ATAC data based on prediction score
    pbmc_atac_filtered = pbmc_atac[pass_threshold, :].copy()

    # Ensure factor levels match for consistent plotting colors
    pbmc_atac_filtered.obs['predicted_id'] = pd.Categorical(
        pbmc_atac_filtered.obs['predicted_id'],
        categories=pbmc_rna.obs['celltype'].cat.categories,
        ordered=True
    )

    # --- Visualize Transferred Labels ---
    print("  Visualizing transferred labels on ATAC UMAP...")
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    sc.pl.umap(
        pbmc_atac_filtered,
        color='predicted_id',
        ax=axes[0],
        show=False,
        title="scATAC-seq Cells (Predicted ID)",
        legend_loc='on data',
        legend_fontsize=8,
        basis='umap_lsi' # Use the LSI-based UMAP
    )
    sc.pl.umap(
        pbmc_rna,
        color='celltype',
        ax=axes[1],
        show=False,
        title="scRNA-seq Cells (Ground Truth)",
        legend_loc='on data',
        legend_fontsize=8
    )

    plt.tight_layout()
    plt.show()


In [ ]:
# --- 8. Co-embedding ---
# The vignette uses TransferData again to impute RNA values into ATAC cells, then merges.
# Scanpy's ingest primarily maps cells and transfers labels.
# A common Scanpy approach for co-visualization is to:
# 1. Concatenate the objects.
# 2. Use a common feature space (e.g., variable genes).
# 3. Run PCA/Neighbors/UMAP on the integrated object, possibly after batch correction (e.g., Harmony, BBKNN).

# Let's follow the vignette's *structure* more closely using ingest's mapping capability.
# We'll project ATAC cells into the RNA's PCA space using ingest, then combine these
# embeddings and run UMAP. This avoids direct imputation but achieves co-visualization.

if pbmc_rna is not None:
    print("\n8. Co-embedding RNA and ATAC data...")

    # 1. Project ATAC activity data into RNA PCA space using ingest (already done partially above)
    # We need the coordinates from `adata_atac_activity_for_ingest.obsm['X_pca']`
    atac_projected_pca = adata_atac_activity_for_ingest.obsm['X_pca']

    # 2. Create a combined AnnData object
    # Ensure both objects use the same index type
    pbmc_rna.obs_names = pbmc_rna.obs_names.astype(str)
    pbmc_atac.obs_names = pbmc_atac.obs_names.astype(str)

    # Select only the common obs (cells) that were processed and potentially filtered
    common_atac_cells = pbmc_atac.obs_names.intersection(adata_atac_activity_for_ingest.obs_names)
    pbmc_atac_subset = pbmc_atac[common_atac_cells, :].copy()
    atac_projected_pca_subset = adata_atac_activity_for_ingest[common_atac_cells, :].obsm['X_pca']

    # Important: Concatenation requires shared variables or handling differences.
    # Since we're primarily interested in using the embeddings, we'll build
    # a new AnnData containing the combined embeddings.

    # Create combined PCA matrix (RNA PCA + ATAC projected PCA)
    combined_pca = np.vstack((pbmc_rna.obsm['X_pca'], atac_projected_pca_subset))

    # Create combined metadata
    obs_rna = pbmc_rna.obs[['tech', 'celltype']].copy()
    obs_atac = pbmc_atac_subset.obs[['tech', 'predicted_id']].copy()
    obs_atac.rename(columns={'predicted_id': 'celltype'}, inplace=True) # Use predicted ID as celltype for ATAC

    combined_obs = pd.concat([obs_rna, obs_atac], axis=0)

    # Create the co-embedding AnnData object
    coembed = ad.AnnData(obs=combined_obs)
    coembed.obsm['X_pca_integrated'] = combined_pca # Store the combined PCA data

    print("  Running UMAP on combined PCA representation...")
    # Run Neighbors and UMAP on the integrated PCA
    sc.pp.neighbors(coembed, use_rep='X_pca_integrated', n_neighbors=30, key_added='coembed_neighbors')
    sc.tl.umap(coembed, min_dist=0.3, neighbors_key='coembed_neighbors')

    # --- Visualize Co-embedding ---
    print("  Visualizing co-embedding...")
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    sc.pl.umap(coembed, color='tech', ax=axes[0], show=False, title="Co-embedding by Technology")
    sc.pl.umap(coembed, color='celltype', ax=axes[1], show=False, title="Co-embedding by Cell Type",
               legend_loc='on data', legend_fontsize=8, na_color='lightgrey') # Show predicted for ATAC

    plt.tight_layout()
    plt.show()

else:
    print("\nSkipping steps 6, 7, 8 because scRNA-seq data could not be loaded.")

print("\nScript Finished.")